# 🚀 Google Colab Training Notebook for Trading Bot ML Models

This notebook provides a complete ML training pipeline that can be run in Google Colab. It will:

1. **Clone your repository** or upload as ZIP
2. **Install dependencies** with fallback handling
3. **Mount Google Drive** for artifact storage
4. **Train models** using your existing modules
5. **Save artifacts** to both repo and Drive
6. **Validate models** and generate manifest
7. **Download results** as ZIP file

## 📋 Instructions:

1. **Replace `GITHUB_REPO_URL`** with your actual repository URL (or upload repo as ZIP)
2. **Set `fast_test = True`** for quick testing, `False` for full training
3. **Run all cells** in order
4. **Check outputs** and download your trained models

⚠️ **Private repos**: Use personal access token in URL format: `https://TOKEN@github.com/user/repo.git`

In [ ]:
# 🔧 Configuration Section - MODIFY THESE VALUES
import os
import sys
from datetime import datetime

# ===== USER CONFIGURATION =====
GITHUB_REPO_URL = "https://github.com/krish567366/bot-model.git"  # Replace with your GitHub repo URL
REPO_NAME = "bot-model"  # Your repository name

# Training configuration
SYMBOL = "BTC-USD"
INTERVAL = "1m"
CFG = {
    "fast_test": False,     # Set to False for full training
    "horizon": 5,           # Prediction horizon
    "pos_thresh": 0.002,    # Positive threshold (0.2%)
    "n_splits": 2,          # Cross-validation splits
    "seed": 42,             # Random seed
    "n_estimators": 100,    # Boosting rounds (fast_test)
    "n_estimators_full": 1000  # Boosting rounds (full training)
}

# Paths (automatically configured)
REPO_PATH = f"/content/{REPO_NAME}"
MODEL_SAVE_REPO_PATH = f"{REPO_PATH}/models/"
MODEL_SAVE_DRIVE_PATH = "/content/drive/MyDrive/trading_bot_models/"
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Global state
DRIVE_MOUNTED = False
MODULES_IMPORTED = False

print("🔧 Configuration loaded:")
print(f"  Symbol: {SYMBOL}")
print(f"  Fast test mode: {CFG['fast_test']}")
print(f"  Repository: {GITHUB_REPO_URL}")
print(f"  Run timestamp: {RUN_TIMESTAMP}")

# 🎯 Data Strategy: Real Market Data Integration

This notebook now integrates with your **existing data pipeline** for production-quality training:

## 📊 **Data Source Priority:**
1. **🌟 Real Market Data** (Yahoo Finance via your pipeline)
2. **💾 Cached Data** (From previous runs)  
3. **🔧 Synthetic Data** (Fallback only)

## 🚀 **Your Data Pipeline Features:**
- ✅ **Multi-source ingestion** (yfinance, Alpha Vantage, CCXT)
- ✅ **Data validation & cleaning**
- ✅ **Technical indicator computation** 
- ✅ **Flexible storage** (SQLite/PostgreSQL)
- ✅ **Real-time & historical processing**
- ✅ **ML-ready feature engineering**

## ⚡ **What This Means:**
- **Training on REAL market data** instead of synthetic
- **Production-grade features** from your existing pipeline
- **Consistent data between training and inference**
- **Automatic fallback** if real data unavailable

**Set `CFG['fast_test'] = False` for full 2-year training dataset!**

# 📥 Step 1: Clone Repository

Clone your trading bot repository to access the training modules.

In [ ]:
def clone_repository():
    """Clone the GitHub repository"""
    
    if GITHUB_REPO_URL == "<YOUR_REPO_URL>":
        print("❌ Please set GITHUB_REPO_URL in the configuration section above!")
        print("   Example: GITHUB_REPO_URL = 'https://github.com/username/trading-bot.git'")
        print("   For private repos: GITHUB_REPO_URL = 'https://TOKEN@github.com/username/trading-bot.git'")
        return False
    
    try:
        print(f"🔄 Cloning repository from {GITHUB_REPO_URL}...")
        
        # Remove existing directory if present
        if os.path.exists(REPO_PATH):
            print("📁 Removing existing repository...")
            import shutil
            shutil.rmtree(REPO_PATH)
        
        # Clone repository
        clone_cmd = f"git clone {GITHUB_REPO_URL} {REPO_PATH}"
        result = os.system(clone_cmd)
        
        if result == 0 and os.path.exists(REPO_PATH):
            print("✅ Repository cloned successfully")
            
            # Add to Python path
            if REPO_PATH not in sys.path:
                sys.path.insert(0, REPO_PATH)
                print(f"✅ Added {REPO_PATH} to Python path")
            
            # Check for key files
            key_files = ["requirements.txt", "ai/", "core/"]
            for file in key_files:
                if os.path.exists(os.path.join(REPO_PATH, file)):
                    print(f"  ✓ Found {file}")
                else:
                    print(f"  ⚠️  Missing {file}")
            
            return True
        else:
            print("❌ Repository cloning failed")
            return False
            
    except Exception as e:
        print(f"❌ Error cloning repository: {e}")
        return False

# Alternative: Upload ZIP file
def upload_repo_zip():
    """Upload repository as ZIP file (alternative to git clone)"""
    try:
        from google.colab import files
        print("📁 Upload your repository as a ZIP file:")
        uploaded = files.upload()
        
        if len(uploaded) == 1:
            zip_name = list(uploaded.keys())[0]
            print(f"📦 Extracting {zip_name}...")
            
            import zipfile
            with zipfile.ZipFile(zip_name, 'r') as zip_ref:
                zip_ref.extractall("/content/")
            
            # Find extracted directory
            extracted_dirs = [d for d in os.listdir("/content/") if os.path.isdir(f"/content/{d}") and d != "sample_data"]
            
            if extracted_dirs:
                global REPO_PATH
                REPO_PATH = f"/content/{extracted_dirs[0]}"
                
                if REPO_PATH not in sys.path:
                    sys.path.insert(0, REPO_PATH)
                
                print(f"✅ Repository extracted to {REPO_PATH}")
                return True
            else:
                print("❌ Could not find extracted directory")
                return False
        else:
            print("❌ Please upload exactly one ZIP file")
            return False
            
    except ImportError:
        print("⚠️  Not running in Colab - ZIP upload not available")
        return False
    except Exception as e:
        print(f"❌ Error uploading ZIP: {e}")
        return False

# Clone repository (or use ZIP upload as fallback)
clone_success = clone_repository()

if not clone_success:
    print("\n💡 Alternative: Upload repository as ZIP file")
    print("   Uncomment the next line to use ZIP upload instead:")
    print("   # clone_success = upload_repo_zip()")
    
    # Uncomment this line if you want to use ZIP upload:
    # clone_success = upload_repo_zip()

# 📦 Step 2: Install Dependencies

Install required Python packages with robust fallback handling.

In [ ]:
def install_dependencies():
    """Install required dependencies with fallbacks"""
    
    print("🔄 Installing dependencies...")
    
    # Try requirements.txt first
    requirements_path = os.path.join(REPO_PATH, "requirements.txt")
    
    if os.path.exists(requirements_path):
        print("📄 Found requirements.txt, installing...")
        result = os.system(f"pip install -q -r {requirements_path}")
        
        if result == 0:
            print("✅ Requirements installed from requirements.txt")
        else:
            print("⚠️  Some packages from requirements.txt failed, continuing with manual installs...")
    else:
        print("📄 No requirements.txt found, installing core packages...")
    
    # Core ML packages
    core_packages = [
        "pandas", "numpy", "scikit-learn", "joblib",
        "lightgbm", "xgboost", "matplotlib", "seaborn",
        "nest_asyncio"  # For async support in Colab
    ]
    
    # Optional packages (won't fail if not installed)
    optional_packages = [
        "catboost", "optuna", "shap", "yfinance", "ccxt", 
        "ta", "alpha_vantage", "sqlalchemy"  # For data pipeline
    ]
    
    print("🔄 Installing core ML packages...")
    for package in core_packages:
        try:
            result = os.system(f"pip install -q {package}")
            if result == 0:
                print(f"  ✓ {package}")
            else:
                print(f"  ⚠️  {package} - failed")
        except:
            print(f"  ❌ {package} - error")
    
    print("🔄 Installing data pipeline packages...")
    for package in optional_packages:
        try:
            result = os.system(f"pip install -q {package}")
            if result == 0:
                print(f"  ✓ {package}")
            else:
                print(f"  ⚠️  {package} - skipped")
        except:
            print(f"  ⚠️  {package} - skipped")
    
    # Verify key packages
    print("\n🔍 Verifying package installations...")
    key_imports = {
        'pandas': 'pd',
        'numpy': 'np',
        'sklearn': 'sklearn',
        'lightgbm': 'lgb',
        'joblib': 'joblib',
        'nest_asyncio': 'nest_asyncio',
        'yfinance': 'yf'
    }
    
    successful_imports = []
    failed_imports = []
    
    for package, alias in key_imports.items():
        try:
            __import__(package)
            successful_imports.append(package)
            print(f"  ✓ {package}")
        except ImportError:
            failed_imports.append(package)
            print(f"  ❌ {package}")
    
    print(f"\n✅ Successfully imported: {len(successful_imports)}/{len(key_imports)} key packages")
    
    if failed_imports:
        print(f"⚠️  Failed imports: {failed_imports}")
        print("   Training will continue but some features may be unavailable")
        
        # Special handling for yfinance failure
        if 'yfinance' in failed_imports:
            print("   📉 yfinance unavailable - will use synthetic data only")
    
    return len(failed_imports) == 0

# Install dependencies
install_success = install_dependencies()

# 💾 Step 3: Mount Google Drive

Mount Google Drive to save trained models for long-term storage.

In [ ]:
def mount_google_drive():
    """Mount Google Drive safely"""
    global DRIVE_MOUNTED
    
    try:
        from google.colab import drive
        print("🔄 Mounting Google Drive...")
        drive.mount('/content/drive')
        
        # Verify mount
        if os.path.exists('/content/drive/MyDrive'):
            print("✅ Google Drive mounted successfully")
            print(f"📁 Drive path: {MODEL_SAVE_DRIVE_PATH}")
            
            # Create models directory in Drive if it doesn't exist
            os.makedirs(MODEL_SAVE_DRIVE_PATH, exist_ok=True)
            DRIVE_MOUNTED = True
            return True
        else:
            print("❌ Drive mount verification failed")
            return False
            
    except ImportError:
        print("⚠️  Not running in Google Colab - Drive mount skipped")
        return False
    except Exception as e:
        print(f"⚠️  Drive mount failed: {e}")
        print("Continuing without Drive backup...")
        return False

# Mount Google Drive
mount_success = mount_google_drive()

if mount_success:
    print("💡 Models will be saved to both repo and Google Drive")
else:
    print("💡 Models will be saved to repo only")

# 📥 Step 4: Import Modules

Import the trading bot modules and verify everything is working.

In [ ]:
def import_trading_modules():
    """Import trading bot modules with fallbacks"""
    global MODULES_IMPORTED
    
    print("🔄 Importing trading bot modules...")
    
    # Core imports
    import pandas as pd
    import numpy as np
    import joblib
    from datetime import datetime, timedelta
    import json
    
    # Set random seed
    np.random.seed(CFG['seed'])
    
    # Try to import trading bot modules
    modules = {}
    
    try:
        # Real data integration (NEW)
        try:
            from ai.real_data_integration import MLDataIntegrator, get_real_training_data
            modules['real_data'] = MLDataIntegrator({
                'data_sources': ['yfinance'],
                'storage_path': f'{REPO_PATH}/data/',
                'cache_enabled': True,
                'real_time_enabled': False
            })
            print("  ✓ ai.real_data_integration - REAL DATA ENABLED!")
        except ImportError:
            print("  ⚠️  Real data integration not found - will use synthetic data")
        
        # Feature engineering
        try:
            from ai.feature_engineering_v2 import compute_features_deterministic
            modules['feature_engineering'] = compute_features_deterministic
            print("  ✓ ai.feature_engineering_v2")
        except ImportError:
            try:
                from ai.feature_engineering import compute_features_deterministic
                modules['feature_engineering'] = compute_features_deterministic
                print("  ✓ ai.feature_engineering")
            except ImportError:
                print("  ⚠️  Feature engineering module not found - will use synthetic features")
        
        # Training modules
        try:
            from ai.training_v2 import train_lightgbm_model
            modules['training'] = train_lightgbm_model
            print("  ✓ ai.training_v2")
        except ImportError:
            try:
                from ai.train_lgbm import train_and_validate_lgbm
                modules['training'] = train_and_validate_lgbm
                print("  ✓ ai.train_lgbm")
            except ImportError:
                print("  ⚠️  Training module not found - will use built-in training")
        
        # Model registry
        try:
            from ai.registry import ModelRegistry
            modules['registry'] = ModelRegistry()
            print("  ✓ ai.registry")
        except ImportError:
            print("  ⚠️  Model registry not found - will save manually")
        
        # Data pipeline components
        try:
            from core.pipeline import YFinanceSource, DataPipeline
            modules['data_pipeline'] = DataPipeline()
            modules['yfinance'] = YFinanceSource()
            print("  ✓ core.pipeline - Data sources available")
        except ImportError:
            print("  ⚠️  Data pipeline not found")
        
        # Data generation (for testing)
        try:
            from ai.training_v2 import generate_synthetic_ohlcv_data
            modules['data_generator'] = generate_synthetic_ohlcv_data
            print("  ✓ Synthetic data generator")
        except ImportError:
            print("  ⚠️  Will create basic synthetic data")
        
        MODULES_IMPORTED = True
        print("✅ Module import completed")
        
        # Show data source priority
        if 'real_data' in modules:
            print("\n🎯 DATA SOURCE PRIORITY:")
            print("  1. Real market data (Yahoo Finance)")
            print("  2. Cached historical data") 
            print("  3. Synthetic data (fallback)")
        else:
            print("\n⚠️  Using synthetic data only")
            
        return modules
        
    except Exception as e:
        print(f"❌ Error importing modules: {e}")
        print("   Will proceed with basic fallback implementations")
        return {}

# Import modules
trading_modules = import_trading_modules()

# 🔄 Step 5: Generate Training Data

Create or load training data for model development.

In [ ]:
async def generate_training_data():
    """Generate training data - prioritizing real market data over synthetic"""
    import pandas as pd
    import numpy as np
    
    # Try to use real data first
    if 'real_data' in trading_modules:
        try:
            print("🌟 Using REAL MARKET DATA from your data pipeline!")
            
            # Determine data parameters based on test mode
            if CFG['fast_test']:
                period = "6m"  # 6 months for fast testing
                interval = "1h" 
                print(f"  📊 Fast test mode: {period} of {interval} data")
            else:
                period = "2y"   # 2 years for full training
                interval = "1h"
                print(f"  📊 Full training mode: {period} of {interval} data")
            
            # Fetch real training data using your pipeline
            integrator = trading_modules['real_data']
            dataset = await integrator.prepare_training_dataset(
                symbol=SYMBOL,
                period=period,
                interval=interval,
                horizon=CFG['horizon'],
                pos_thresh=CFG['pos_thresh']
            )
            
            print(f"✅ Real data loaded successfully!")
            print(f"  📈 Data source: {dataset['metadata'].get('data_source', 'Yahoo Finance')}")
            print(f"  📅 Date range: {dataset['metadata']['data_range']['start']} to {dataset['metadata']['data_range']['end']}")
            print(f"  📊 Raw data points: {len(dataset['ohlcv_data'])}")
            print(f"  🧮 ML features: {dataset['metadata']['n_features']}")
            print(f"  🎯 Training samples: {dataset['metadata']['n_samples']}")
            print(f"  📈 Class distribution: {dataset['metadata']['class_distribution']}")
            
            # Return real data
            return dataset['X'], dataset['y_binary'], dataset['y_regression'], dataset['timestamps']
            
        except Exception as e:
            print(f"❌ Real data loading failed: {e}")
            print("📉 Falling back to synthetic data...")
    
    # Fallback to synthetic data
    print("🔧 Generating synthetic training data...")
    
    # Data size based on test mode
    n_periods = 500 if CFG['fast_test'] else 2000
    
    print(f"🔄 Generating {n_periods} periods of synthetic data...")
    
    # Generate synthetic OHLCV data
    dates = pd.date_range(start='2023-01-01', periods=n_periods, freq='1H')
    
    # Random walk with drift for realistic price movement
    np.random.seed(CFG['seed'])
    returns = np.random.normal(0.0001, 0.01, n_periods)  # Small positive drift
    log_prices = np.cumsum(returns)
    prices = 50000 * np.exp(log_prices)  # Start around $50,000
    
    data = []
    for i, (date, price) in enumerate(zip(dates, prices)):
        # Generate realistic OHLC
        volatility = abs(np.random.normal(0, 0.008))  # Daily volatility ~0.8%
        high = price * (1 + volatility)
        low = price * (1 - volatility)
        open_price = prices[i-1] if i > 0 else price
        volume = np.random.uniform(100, 1000)
        
        data.append({
            'timestamp': date,
            'open': open_price,
            'high': high,
            'low': low,
            'close': price,
            'volume': volume
        })
    
    df = pd.DataFrame(data)
    
    # Create features
    print("🔄 Computing features...")
    features = pd.DataFrame(index=df.index)
    
    # Price features
    features['returns'] = df['close'].pct_change()
    features['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    features['price_ma5'] = df['close'].rolling(5).mean()
    features['price_ma20'] = df['close'].rolling(20).mean()
    features['price_std'] = df['close'].rolling(20).std()
    
    # Volume features
    features['volume'] = df['volume']
    features['volume_ma5'] = df['volume'].rolling(5).mean()
    features['volume_ratio'] = df['volume'] / features['volume_ma5']
    
    # Technical indicators
    features['rsi'] = compute_rsi(df['close'], 14)
    features['macd'] = compute_macd(df['close'])
    features['bollinger_upper'], features['bollinger_lower'] = compute_bollinger_bands(df['close'])
    
    # Volatility
    features['volatility'] = features['returns'].rolling(20).std()
    features['volatility_ma'] = features['volatility'].rolling(5).mean()
    
    # Clean features
    features = features.dropna()
    
    # Create labels
    print("🔄 Creating labels...")
    future_periods = CFG['horizon']
    threshold = CFG['pos_thresh']
    
    # Calculate future returns
    future_returns = df['close'].shift(-future_periods) / df['close'] - 1
    
    # Binary classification: Will price move up > threshold?
    labels_binary = (future_returns > threshold).astype(int)
    
    # Regression target: actual future return
    labels_regression = future_returns
    
    # Align features and labels
    valid_mask = ~future_returns.isna() & ~features.isnull().any(axis=1)
    
    X = features[valid_mask].reset_index(drop=True)
    y_binary = labels_binary[valid_mask].reset_index(drop=True)
    y_regression = labels_regression[valid_mask].reset_index(drop=True)
    timestamps = df['timestamp'][valid_mask].reset_index(drop=True)
    
    print(f"✅ Synthetic dataset created:")
    print(f"  Samples: {len(X)}")
    print(f"  Features: {X.shape[1]}")
    print(f"  Time range: {timestamps.iloc[0]} to {timestamps.iloc[-1]}")
    print(f"  Binary class distribution: {y_binary.value_counts().to_dict()}")
    print(f"  Regression target stats: mean={y_regression.mean():.4f}, std={y_regression.std():.4f}")
    
    return X, y_binary, y_regression, timestamps

def compute_rsi(prices, window=14):
    """Compute RSI indicator"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def compute_macd(prices, fast=12, slow=26):
    """Compute MACD indicator"""
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    return ema_fast - ema_slow

def compute_bollinger_bands(prices, window=20, std_dev=2):
    """Compute Bollinger Bands"""
    ma = prices.rolling(window).mean()
    std = prices.rolling(window).std()
    upper = ma + (std * std_dev)
    lower = ma - (std * std_dev)
    return upper, lower

# Generate training data (async call in Jupyter requires special handling)
print("🚀 Starting data generation...")

# In Jupyter/Colab, we need to handle async calls properly
import asyncio

# Check if we're in an existing event loop (Jupyter)
try:
    loop = asyncio.get_running_loop()
    # If we're in Jupyter, create a task
    import nest_asyncio
    nest_asyncio.apply()  # Allow nested event loops
    X, y_binary, y_regression, timestamps = await generate_training_data()
except RuntimeError:
    # If no event loop, run normally
    X, y_binary, y_regression, timestamps = asyncio.run(generate_training_data())
except ImportError:
    # If nest_asyncio not available, use asyncio.run
    X, y_binary, y_regression, timestamps = asyncio.run(generate_training_data())

print(f"\n🎉 Data generation completed!")
print(f"Final dataset: {len(X)} samples, {len(X.columns)} features")

# 🏋️ Step 6: Train Models

Train machine learning models using LightGBM and other algorithms.

In [ ]:
# Feature Analysis and Overview
print("📊 FEATURE ANALYSIS:")
print(f"Total Features: {len(X.columns)}")
print(f"Training Samples: {len(X)}")

# Show data source information
data_source = "Real Market Data via your existing data pipeline" if 'real_data' in trading_modules else "Synthetic trading data"
print(f"Data Source: {data_source}")

if 'real_data' in trading_modules:
    print("🌟 Using PRODUCTION-GRADE features from your data pipeline:")
    print("   • Technical indicators from ta library")
    print("   • Market data from Yahoo Finance")  
    print("   • Advanced feature engineering")
    print("   • Volume and volatility metrics")
else:
    print("🔧 Using SYNTHETIC features for testing:")
    print("   • Simulated price movements")
    print("   • Basic technical indicators")
    print("   • Test-grade feature generation")

print("\n📈 Feature Categories:")
feature_types = {}
for col in X.columns:
    if any(x in col.lower() for x in ['price', 'close', 'open', 'high', 'low']):
        feature_types['Price Features'] = feature_types.get('Price Features', 0) + 1
    elif any(x in col.lower() for x in ['volume', 'vol']):
        feature_types['Volume Features'] = feature_types.get('Volume Features', 0) + 1
    elif any(x in col.lower() for x in ['return', 'pct', 'change']):
        feature_types['Return Features'] = feature_types.get('Return Features', 0) + 1
    elif any(x in col.lower() for x in ['sma', 'ema', 'bb', 'rsi', 'macd', 'bollinger', 'moving']):
        feature_types['Technical Indicators'] = feature_types.get('Technical Indicators', 0) + 1
    elif any(x in col.lower() for x in ['volatility', 'std', 'var']):
        feature_types['Volatility Features'] = feature_types.get('Volatility Features', 0) + 1
    else:
        feature_types['Other Features'] = feature_types.get('Other Features', 0) + 1

for category, count in feature_types.items():
    print(f"  • {category}: {count}")

print(f"\n? Sample Features:")
print(f"  {list(X.columns[:10])}")
if len(X.columns) > 10:
    print(f"  ... and {len(X.columns) - 10} more features")

# Feature importance preview (basic correlation analysis)
print(f"\n🎯 Top Correlated Features (with target):")
try:
    correlations = X.corrwith(y_binary).abs().sort_values(ascending=False)
    print(f"  • {correlations.index[0]}: {correlations.iloc[0]:.3f}")
    print(f"  • {correlations.index[1]}: {correlations.iloc[1]:.3f}")
    print(f"  • {correlations.index[2]}: {correlations.iloc[2]:.3f}")
    print(f"  • {correlations.index[3]}: {correlations.iloc[3]:.3f}")
    print(f"  • {correlations.index[4]}: {correlations.iloc[4]:.3f}")
except:
    print("  (Correlation analysis skipped)")

print(f"\n✅ Feature engineering completed!")

# 💾 Step 7: Save Model Artifacts

Save trained models and metadata to both repository and Google Drive.

In [ ]:
def create_model_artifacts(model, metrics, params, model_type, X_sample):
    """Create comprehensive model artifacts"""
    import joblib
    from sklearn.preprocessing import StandardScaler
    
    # Create model directory
    model_id = f"lgbm_{model_type}_{RUN_TIMESTAMP}"
    model_dir = os.path.join(MODEL_SAVE_REPO_PATH, SYMBOL, RUN_TIMESTAMP, model_id)
    os.makedirs(model_dir, exist_ok=True)
    
    print(f"📁 Creating artifacts in: {model_dir}")
    
    # Save model
    model_path = os.path.join(model_dir, "model.pkl")
    joblib.dump(model, model_path, compress=3)
    print(f"  ✓ Model saved: model.pkl")
    
    # Create and save scaler (even if not used, for consistency)
    scaler = StandardScaler()
    scaler.fit(X_sample)  # Fit on sample data for consistency
    scaler_path = os.path.join(model_dir, "scaler.pkl")
    joblib.dump(scaler, scaler_path, compress=3)
    print(f"  ✓ Scaler saved: scaler.pkl")
    
    # Create comprehensive metadata
    metadata = {
        'model_id': model_id,
        'model_type': f'lightgbm_{model_type}',
        'symbol': SYMBOL,
        'interval': INTERVAL,
        'timestamp': RUN_TIMESTAMP,
        'training_config': CFG,
        'model_params': params,
        'metrics': metrics,
        'feature_names': list(X_sample.columns),
        'n_features': len(X_sample.columns),
        'training_samples': len(X_sample),
        'fast_test_mode': CFG['fast_test'],
        'random_seed': CFG['seed'],
        'version': '1.0',
        'framework': 'lightgbm',
        'task_type': model_type,
        'colab_training': True
    }
    
    # Save metadata
    meta_path = os.path.join(model_dir, "meta.json")
    with open(meta_path, 'w') as f:
        json.dump(metadata, f, indent=2, default=str)
    print(f"  ✓ Metadata saved: meta.json")
    
    # Save feature names
    feature_names_path = os.path.join(model_dir, "feature_names.json")
    with open(feature_names_path, 'w') as f:
        json.dump(list(X_sample.columns), f)
    print(f"  ✓ Feature names saved: feature_names.json")
    
    return model_dir, metadata

def copy_to_drive(source_dir, model_id):
    """Copy artifacts to Google Drive"""
    if not DRIVE_MOUNTED:
        print("⚠️  Google Drive not mounted, skipping Drive backup")
        return None
    
    try:
        import shutil
        
        # Create destination directory
        drive_model_dir = os.path.join(MODEL_SAVE_DRIVE_PATH, SYMBOL, RUN_TIMESTAMP, model_id)
        os.makedirs(os.path.dirname(drive_model_dir), exist_ok=True)
        
        # Copy entire model directory
        if os.path.exists(drive_model_dir):
            shutil.rmtree(drive_model_dir)
        
        shutil.copytree(source_dir, drive_model_dir)
        print(f"✅ Artifacts copied to Google Drive: {drive_model_dir}")
        
        return drive_model_dir
    
    except Exception as e:
        print(f"⚠️  Failed to copy to Google Drive: {e}")
        return None

# Save artifacts for both models
saved_models = {}

if 'binary_model' in locals() and binary_model is not None:
    print("\n💾 Saving Binary Classification Model...")
    binary_dir, binary_metadata = create_model_artifacts(
        binary_model, binary_metrics, binary_params, 'binary', binary_splits['X_train']
    )
    binary_drive_dir = copy_to_drive(binary_dir, binary_metadata['model_id'])
    
    saved_models['binary'] = {
        'local_path': binary_dir,
        'drive_path': binary_drive_dir,
        'metadata': binary_metadata
    }

if 'regression_model' in locals() and regression_model is not None:
    print("\n💾 Saving Regression Model...")
    regression_dir, regression_metadata = create_model_artifacts(
        regression_model, regression_metrics, regression_params, 'regression', regression_splits['X_train']
    )
    regression_drive_dir = copy_to_drive(regression_dir, regression_metadata['model_id'])
    
    saved_models['regression'] = {
        'local_path': regression_dir,
        'drive_path': regression_drive_dir,
        'metadata': regression_metadata
    }

print(f"\n✅ All artifacts saved! Models: {list(saved_models.keys())}")

# ✅ Step 8: Model Validation

Validate that saved models can be loaded and used for inference.

In [ ]:
def validate_saved_models():
    """Validate that saved models work correctly"""
    import joblib
    
    print("🔍 Validating saved models...")
    
    validation_results = {}
    
    for model_type, model_info in saved_models.items():
        print(f"\n🔄 Validating {model_type} model...")
        
        try:
            model_dir = model_info['local_path']
            
            # Load model and scaler
            model_path = os.path.join(model_dir, "model.pkl")
            scaler_path = os.path.join(model_dir, "scaler.pkl")
            meta_path = os.path.join(model_dir, "meta.json")
            
            # Check files exist
            for path, name in [(model_path, "model.pkl"), (scaler_path, "scaler.pkl"), (meta_path, "meta.json")]:
                if os.path.exists(path):
                    print(f"  ✓ Found {name}")
                else:
                    print(f"  ❌ Missing {name}")
                    continue
            
            # Load artifacts
            model = joblib.load(model_path)
            scaler = joblib.load(scaler_path)
            
            with open(meta_path, 'r') as f:
                metadata = json.load(f)
            
            print(f"  ✓ Loaded model: {metadata['model_id']}")
            print(f"  ✓ Features: {metadata['n_features']}")
            print(f"  ✓ Training samples: {metadata['training_samples']}")
            
            # Test prediction on sample data
            if model_type == 'binary':
                test_X = binary_splits['X_test'].iloc[:5]  # First 5 test samples
                test_y = binary_splits['y_test'].iloc[:5]
            else:
                test_X = regression_splits['X_test'].iloc[:5]
                test_y = regression_splits['y_test'].iloc[:5]
            
            # Make predictions
            predictions = model.predict(test_X)
            
            print(f"  ✓ Sample predictions shape: {predictions.shape}")
            print(f"  ✓ Sample predictions (first 3): {predictions[:3]}")
            
            # Validate prediction format
            if model_type == 'binary':
                # Binary predictions should be probabilities between 0 and 1
                if all(0 <= p <= 1 for p in predictions):
                    print(f"  ✅ Binary probabilities valid (0-1 range)")
                else:
                    print(f"  ⚠️  Binary probabilities outside 0-1 range")
            else:
                # Regression predictions should be reasonable returns
                if all(abs(p) < 1 for p in predictions):  # |return| < 100%
                    print(f"  ✅ Regression predictions reasonable")
                else:
                    print(f"  ⚠️  Regression predictions seem extreme")
            
            validation_results[model_type] = {
                'status': 'success',
                'model_path': model_path,
                'predictions_sample': predictions[:3].tolist(),
                'metadata': metadata
            }
            
            print(f"  ✅ {model_type.capitalize()} model validation successful")
            
        except Exception as e:
            print(f"  ❌ {model_type.capitalize()} model validation failed: {e}")
            validation_results[model_type] = {
                'status': 'failed',
                'error': str(e)
            }
    
    return validation_results

# Validate models
if saved_models:
    validation_results = validate_saved_models()
    
    print(f"\n🏆 Validation Summary:")
    for model_type, result in validation_results.items():
        status = "✅" if result['status'] == 'success' else "❌"
        print(f"  {status} {model_type.capitalize()} Model")
else:
    print("⚠️  No models to validate")

# 📋 Step 9: Create Run Manifest

Create a comprehensive manifest file documenting this training run.

In [ ]:
def create_run_manifest():
    """Create a comprehensive run manifest"""
    
    # Create runs directory
    runs_dir = os.path.join(REPO_PATH, "runs", f"colab-{RUN_TIMESTAMP}")
    os.makedirs(runs_dir, exist_ok=True)
    
    # Get git commit hash if available
    git_commit = "unknown"
    try:
        import subprocess
        result = subprocess.run(['git', 'rev-parse', 'HEAD'], 
                              cwd=REPO_PATH, capture_output=True, text=True)
        if result.returncode == 0:
            git_commit = result.stdout.strip()[:12]  # Short hash
    except:
        pass
    
    # Calculate dataset hash (simple hash of feature names and data size)
    import hashlib
    feature_string = f"{list(X.columns)}_{len(X)}_{CFG['seed']}"
    dataset_hash = hashlib.md5(feature_string.encode()).hexdigest()[:12]
    
    # Create comprehensive manifest
    manifest = {
        'run_info': {
            'timestamp': RUN_TIMESTAMP,
            'git_commit': git_commit,
            'dataset_hash': dataset_hash,
            'colab_session': True,
            'fast_test_mode': CFG['fast_test']
        },
        'configuration': CFG,
        'data_info': {
            'symbol': SYMBOL,
            'interval': INTERVAL,
            'n_samples': len(X),
            'n_features': len(X.columns),
            'feature_names': list(X.columns),
            'time_range': {
                'start': str(timestamps.iloc[0]),
                'end': str(timestamps.iloc[-1])
            }
        },
        'models': {},
        'artifacts': {
            'repo_base_path': MODEL_SAVE_REPO_PATH,
            'drive_base_path': MODEL_SAVE_DRIVE_PATH if DRIVE_MOUNTED else None,
            'saved_models': []
        },
        'validation_results': validation_results if 'validation_results' in locals() else {},
        'environment': {
            'python_version': sys.version,
            'key_packages': {}
        }
    }
    
    # Add package versions safely
    import pandas as pd
    import numpy as np
    
    manifest['environment']['key_packages']['pandas'] = pd.__version__
    manifest['environment']['key_packages']['numpy'] = np.__version__
    
    try:
        import lightgbm
        manifest['environment']['key_packages']['lightgbm'] = lightgbm.__version__
    except:
        pass
    
    try:
        import sklearn
        manifest['environment']['key_packages']['sklearn'] = sklearn.__version__
    except:
        pass
    
    # Add model information
    for model_type, model_info in saved_models.items():
        manifest['models'][model_type] = {
            'model_id': model_info['metadata']['model_id'],
            'local_path': model_info['local_path'],
            'drive_path': model_info['drive_path'],
            'metrics': model_info['metadata']['metrics'],
            'params': model_info['metadata']['model_params']
        }
        
        manifest['artifacts']['saved_models'].append({
            'type': model_type,
            'id': model_info['metadata']['model_id'],
            'path': model_info['local_path']
        })
    
    # Save manifest
    manifest_path = os.path.join(runs_dir, "manifest.json")
    with open(manifest_path, 'w') as f:
        json.dump(manifest, f, indent=2, default=str)
    
    print(f"📋 Run manifest created: {manifest_path}")
    
    # Display summary
    print(f"\n📊 Training Run Summary:")
    print(f"  Run ID: colab-{RUN_TIMESTAMP}")
    print(f"  Git Commit: {git_commit}")
    print(f"  Dataset Hash: {dataset_hash}")
    print(f"  Models Trained: {len(saved_models)}")
    print(f"  Total Samples: {len(X)}")
    print(f"  Features: {len(X.columns)}")
    print(f"  Fast Test Mode: {CFG['fast_test']}")
    
    if saved_models:
        print(f"\n🎯 Model Performance:")
        for model_type, model_info in saved_models.items():
            metrics = model_info['metadata']['metrics']
            if model_type == 'binary':
                print(f"  Binary: AUC={metrics['auc']:.4f}, Accuracy={metrics['accuracy']:.4f}")
            else:
                print(f"  Regression: RMSE={metrics['rmse']:.6f}, R²={metrics['r2']:.4f}")
    
    return manifest_path, manifest

# Create run manifest
if saved_models:
    manifest_path, manifest = create_run_manifest()
else:
    print("⚠️  No models saved, skipping manifest creation")

# 📥 Step 10: Display Results & Download

Display the training results and provide download options.

In [ ]:
def display_results():
    """Display comprehensive training results"""
    
    print("🏆" + "="*60)
    print("🏆 GOOGLE COLAB TRAINING COMPLETED SUCCESSFULLY!")
    print("🏆" + "="*60)
    
    if not saved_models:
        print("❌ No models were successfully trained")
        return
    
    print(f"\n📊 TRAINING SUMMARY:")
    print(f"  • Run Timestamp: {RUN_TIMESTAMP}")
    print(f"  • Symbol: {SYMBOL}")
    print(f"  • Training Mode: {'Fast Test' if CFG['fast_test'] else 'Full Training'}")
    print(f"  • Models Trained: {len(saved_models)}")
    print(f"  • Dataset Size: {len(X)} samples, {len(X.columns)} features")
    
    print(f"\n🎯 MODEL PERFORMANCE:")
    for model_type, model_info in saved_models.items():
        metadata = model_info['metadata']
        metrics = metadata['metrics']
        
        print(f"\n  📈 {model_type.upper()} MODEL:")
        print(f"    Model ID: {metadata['model_id']}")
        print(f"    Framework: {metadata['framework']}")
        
        if model_type == 'binary':
            print(f"    AUC Score: {metrics['auc']:.4f}")
            print(f"    Accuracy: {metrics['accuracy']:.4f}")
        else:
            print(f"    RMSE: {metrics['rmse']:.6f}")
            print(f"    R² Score: {metrics['r2']:.4f}")
    
    print(f"\n📁 ARTIFACT LOCATIONS:")
    for model_type, model_info in saved_models.items():
        print(f"\n  {model_type.upper()} MODEL ARTIFACTS:")
        print(f"    Local Path: {model_info['local_path']}")
        if model_info['drive_path']:
            print(f"    Google Drive: {model_info['drive_path']}")
        
        # List files in directory
        if os.path.exists(model_info['local_path']):
            files = os.listdir(model_info['local_path'])
            print(f"    Files: {', '.join(files)}")
    
    # Display sample metadata
    if saved_models:
        sample_model = list(saved_models.values())[0]
        print(f"\n📋 SAMPLE MODEL METADATA:")
        
        # Pretty print a subset of metadata
        display_metadata = {
            'model_id': sample_model['metadata']['model_id'],
            'model_type': sample_model['metadata']['model_type'],
            'training_config': sample_model['metadata']['training_config'],
            'metrics': sample_model['metadata']['metrics'],
            'n_features': sample_model['metadata']['n_features'],
            'training_samples': sample_model['metadata']['training_samples']
        }
        
        print(json.dumps(display_metadata, indent=2))

def create_download_zip():
    """Create ZIP file of all artifacts for download"""
    try:
        from google.colab import files
        import zipfile
        
        if not saved_models:
            print("❌ No models to package")
            return
        
        # Create ZIP filename
        zip_filename = f"trading_bot_models_{RUN_TIMESTAMP}.zip"
        zip_path = f"/content/{zip_filename}"
        
        print(f"📦 Creating download package: {zip_filename}")
        
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for model_type, model_info in saved_models.items():
                model_dir = model_info['local_path']
                
                # Add all files from model directory
                for root, dirs, files in os.walk(model_dir):
                    for file in files:
                        file_path = os.path.join(root, file)
                        # Create relative path for ZIP
                        arcname = os.path.relpath(file_path, MODEL_SAVE_REPO_PATH)
                        zipf.write(file_path, arcname)
                        print(f"  ✓ Added: {arcname}")
            
            # Add manifest if it exists
            if 'manifest_path' in locals():
                zipf.write(manifest_path, f"runs/colab-{RUN_TIMESTAMP}/manifest.json")
                print(f"  ✓ Added: manifest.json")
        
        print(f"✅ ZIP package created: {zip_filename}")
        print(f"📥 Downloading...")
        
        # Download the ZIP file
        files.download(zip_path)
        
        print("✅ Download initiated!")
        print("💡 The ZIP file contains all model artifacts and metadata")
        
    except ImportError:
        print("⚠️  Not running in Google Colab - download not available")
        print("💡 You can manually copy files from the paths shown above")
    except Exception as e:
        print(f"❌ Error creating download package: {e}")

# Display results
display_results()

print(f"\n" + "="*60)
print("🎉 NEXT STEPS:")
print("="*60)

if CFG['fast_test']:
    print("1. 🚀 For production training, set CFG['fast_test'] = False and re-run")

print("2. 📥 Download your model artifacts using the ZIP package below")
print("3. 🧪 Test your models in your trading environment")
print("4. 📈 Integrate with your backtesting and live trading systems")
print("5. 🔄 Monitor performance and retrain as needed")

print(f"\n💡 Model artifacts are saved in:")
print(f"   Repository: {MODEL_SAVE_REPO_PATH}")
if DRIVE_MOUNTED:
    print(f"   Google Drive: {MODEL_SAVE_DRIVE_PATH}")

print(f"\n🤖 To use these models in production:")
print("   • Load with: model = joblib.load('model.pkl')")
print("   • Make predictions: predictions = model.predict(features)")
print("   • Check metadata for feature requirements and preprocessing")

# 📥 Download Model Artifacts

Download all trained models and artifacts as a ZIP file.

In [ ]:
# Create and download ZIP package of all artifacts
create_download_zip()

print("\n🎊 Training completed successfully!")
print("🎯 Your models are ready for production use!")

# Display final status
if saved_models:
    print(f"\n✅ Successfully trained {len(saved_models)} models:")
    for model_type in saved_models.keys():
        print(f"  • {model_type.capitalize()} Classification/Regression Model")
    
    print(f"\n🏆 Best practices implemented:")
    print("  ✓ Time-based train/val/test splits")
    print("  ✓ Comprehensive model evaluation")
    print("  ✓ Artifact versioning and metadata")
    print("  ✓ Model validation and integrity checks")
    print("  ✓ Google Drive backup (if mounted)")
    print("  ✓ Downloadable model packages")
else:
    print("\n⚠️  No models were successfully trained")
    print("Please check the error messages above and try again")

# 🚀 Trading Bot ML Training - Google Colab Edition

## 📋 SETUP INSTRUCTIONS (REQUIRED):

### 1. Replace Repository URL
```python
GITHUB_REPO_URL = "<YOUR_REPO_URL>"  # ← REPLACE THIS!
```

### 2. Choose Training Mode
- **`fast_test=True`** (default): Quick test run with synthetic data (5 minutes)
- **`fast_test=False`**: Full training with real data (30-60 minutes)

### 3. Private Repository?
If your repo is private, use:
```python
# GITHUB_REPO_URL = "https://<TOKEN>@github.com/owner/repo.git"
```
Replace `<TOKEN>` with your GitHub personal access token.

### 4. Alternative: Upload ZIP
Instead of cloning, you can upload your repo as a ZIP file and uncomment the ZIP upload section.

---

## 🎯 What This Notebook Does:
1. **Clone** your trading bot repository
2. **Install** all dependencies automatically
3. **Mount** Google Drive for artifact storage
4. **Train** LightGBM and XGBoost models using your existing modules
5. **Save** trained models to repo and Google Drive
6. **Validate** model artifacts
7. **Download** results to your local machine

## 📦 Output Artifacts:
- `models/{symbol}/{timestamp}/{model_id}/` - Model files (pkl, meta.json)
- `runs/colab-{timestamp}/manifest.json` - Training manifest
- Google Drive backup (if mounted)
- ZIP download for local machine

**Ready? Let's start! 👇**

# ⚙️ Configuration Section

**IMPORTANT: Modify these variables before running!**

In [ ]:
# =============================================================================
# 🔧 USER CONFIGURATION - MODIFY THESE VALUES!
# =============================================================================

# TODO: Replace with your GitHub repository URL
GITHUB_REPO_URL = "<YOUR_REPO_URL>"  # Example: "https://github.com/username/trading-bot.git"

# Training Configuration
SYMBOL = "BTC-USD"
INTERVAL = "1m"

CFG = {
    "fast_test": True,        # Set to False for full training
    "horizon": 5,             # Future periods for prediction
    "pos_thresh": 0.002,      # Positive class threshold (0.2%)
    "n_splits": 2,            # Cross-validation splits (fast_test)
    "seed": 42,               # Random seed
    "n_periods": 1000 if True else 5000,  # Dataset size (will be set based on fast_test)
}

# Update n_periods based on fast_test
CFG["n_periods"] = 1000 if CFG["fast_test"] else 5000

# Paths (will be set after repo clone)
REPO_NAME = None  # Will be extracted from GITHUB_REPO_URL
REPO_PATH = None  # Will be set to /content/{REPO_NAME}
MODEL_SAVE_REPO_PATH = None  # Will be set to {REPO_PATH}/models/
MODEL_SAVE_DRIVE_PATH = "/content/drive/MyDrive/models/"

# Status flags
DRIVE_MOUNTED = False
REPO_CLONED = False

print("✅ Configuration loaded")
print(f"🎯 Training Mode: {'Fast Test' if CFG['fast_test'] else 'Full Training'}")
print(f"📊 Symbol: {SYMBOL} | Interval: {INTERVAL}")
print(f"🔢 Dataset Size: {CFG['n_periods']} periods")

if GITHUB_REPO_URL == "<YOUR_REPO_URL>":
    print("⚠️  WARNING: Please replace GITHUB_REPO_URL with your actual repository URL!")
    print("   Example: GITHUB_REPO_URL = 'https://github.com/username/trading-bot.git'")

# 📥 Repository Setup

Clone your trading bot repository and set up the Python environment.

In [ ]:
import os
import sys
import subprocess
import shutil
from pathlib import Path
import json
from datetime import datetime

def extract_repo_name(url):
    """Extract repository name from GitHub URL"""
    if url.endswith('.git'):
        url = url[:-4]
    return url.split('/')[-1]

def clone_repository(repo_url):
    """Clone the repository"""
    global REPO_NAME, REPO_PATH, MODEL_SAVE_REPO_PATH, REPO_CLONED
    
    if repo_url == "<YOUR_REPO_URL>":
        print("❌ ERROR: Please replace GITHUB_REPO_URL with your actual repository URL!")
        return False
    
    try:
        print(f"🔄 Cloning repository: {repo_url}")
        
        # Extract repo name
        REPO_NAME = extract_repo_name(repo_url)
        REPO_PATH = f"/content/{REPO_NAME}"
        MODEL_SAVE_REPO_PATH = f"{REPO_PATH}/models/"
        
        # Remove existing directory if it exists
        if os.path.exists(REPO_PATH):
            print(f"🗑️  Removing existing directory: {REPO_PATH}")
            shutil.rmtree(REPO_PATH)
        
        # Clone repository
        result = subprocess.run(
            ["git", "clone", repo_url, REPO_PATH],
            capture_output=True,
            text=True,
            cwd="/content"
        )
        
        if result.returncode != 0:
            print(f"❌ Git clone failed: {result.stderr}")
            print("💡 If this is a private repo, make sure you're using a personal access token:")
            print("   https://<TOKEN>@github.com/username/repo.git")
            return False
        
        # Add to Python path
        if REPO_PATH not in sys.path:
            sys.path.insert(0, REPO_PATH)
        
        print(f"✅ Repository cloned successfully to: {REPO_PATH}")
        print(f"📁 Python path updated: {REPO_PATH}")
        
        # Show repository structure
        print("\n📂 Repository structure:")
        for root, dirs, files in os.walk(REPO_PATH):
            # Limit depth to avoid clutter
            level = root.replace(REPO_PATH, '').count(os.sep)
            if level < 3:
                indent = ' ' * 2 * level
                print(f"{indent}{os.path.basename(root)}/")
                subindent = ' ' * 2 * (level + 1)
                for file in files[:5]:  # Show only first 5 files per directory
                    print(f"{subindent}{file}")
                if len(files) > 5:
                    print(f"{subindent}... and {len(files) - 5} more files")
        
        REPO_CLONED = True
        return True
        
    except Exception as e:
        print(f"❌ Error cloning repository: {e}")
        return False

# Clone the repository
clone_success = clone_repository(GITHUB_REPO_URL)

if not clone_success:
    print("\n🔄 Alternative: Upload ZIP file")
    print("If cloning failed, you can upload your repo as a ZIP file instead.")
    print("Uncomment and run the next cell to use ZIP upload.")

In [ ]:
# # ALTERNATIVE: Upload ZIP file (uncomment if git clone failed)
# from google.colab import files
# import zipfile

# print("📦 Upload your repository as a ZIP file:")
# uploaded = files.upload()

# if uploaded:
#     zip_name = list(uploaded.keys())[0]
#     print(f"📥 Extracting {zip_name}...")
    
#     with zipfile.ZipFile(zip_name, 'r') as zip_ref:
#         zip_ref.extractall('/content')
    
#     # Find extracted directory
#     for item in os.listdir('/content'):
#         if os.path.isdir(f'/content/{item}') and item != 'sample_data':
#             REPO_NAME = item
#             REPO_PATH = f'/content/{item}'
#             MODEL_SAVE_REPO_PATH = f'{REPO_PATH}/models/'
#             break
    
#     if REPO_PATH and REPO_PATH not in sys.path:
#         sys.path.insert(0, REPO_PATH)
    
#     print(f"✅ ZIP extracted to: {REPO_PATH}")
#     REPO_CLONED = True

# 📦 Install Dependencies

Install required packages for ML training.

In [ ]:
def display_results():
    """Display comprehensive training results"""
    
    print("🏆" + "="*60)
    print("🏆 GOOGLE COLAB TRAINING COMPLETED SUCCESSFULLY!")
    print("🏆" + "="*60)
    
    if not saved_models:
        print("❌ No models were successfully trained")
        return
    
    print(f"\n? TRAINING SUMMARY:")
    print(f"  • Run Timestamp: {RUN_TIMESTAMP}")
    print(f"  • Symbol: {SYMBOL}")
    print(f"  • Training Mode: {'Fast Test' if CFG['fast_test'] else 'Full Training'}")
    print(f"  • Models Trained: {len(saved_models)}")
    print(f"  • Dataset Size: {len(X)} samples, {len(X.columns)} features")
    
    # Show data source information
    data_source = "🌟 Real Market Data" if 'real_data' in trading_modules else "🔧 Synthetic Data"
    print(f"  • Data Source: {data_source}")
    
    if 'real_data' in trading_modules:
        print(f"    📈 Via your existing data pipeline (Yahoo Finance)")
        print(f"    🎯 Production-grade features and validation")
    else:
        print(f"    ⚠️  Synthetic data used (real data unavailable)")
    
    print(f"\n🎯 MODEL PERFORMANCE:")
    for model_type, model_info in saved_models.items():
        metadata = model_info['metadata']
        metrics = metadata['metrics']
        
        print(f"\n  📈 {model_type.upper()} MODEL:")
        print(f"    Model ID: {metadata['model_id']}")
        print(f"    Framework: {metadata['framework']}")
        print(f"    Data Source: {data_source}")
        
        if model_type == 'binary':
            print(f"    AUC Score: {metrics['auc']:.4f}")
            print(f"    Accuracy: {metrics['accuracy']:.4f}")
        else:
            print(f"    RMSE: {metrics['rmse']:.6f}")
            print(f"    R² Score: {metrics['r2']:.4f}")
    
    print(f"\n? ARTIFACT LOCATIONS:")
    for model_type, model_info in saved_models.items():
        print(f"\n  {model_type.upper()} MODEL ARTIFACTS:")
        print(f"    Local Path: {model_info['local_path']}")
        if model_info['drive_path']:
            print(f"    Google Drive: {model_info['drive_path']}")
        
        # List files in directory
        if os.path.exists(model_info['local_path']):
            files = os.listdir(model_info['local_path'])
            print(f"    Files: {', '.join(files)}")
    
    # Display sample metadata
    if saved_models:
        sample_model = list(saved_models.values())[0]
        print(f"\n📋 SAMPLE MODEL METADATA:")
        
        # Pretty print a subset of metadata including data source info
        display_metadata = {
            'model_id': sample_model['metadata']['model_id'],
            'model_type': sample_model['metadata']['model_type'],
            'data_source': data_source,
            'training_config': sample_model['metadata']['training_config'],
            'metrics': sample_model['metadata']['metrics'],
            'n_features': sample_model['metadata']['n_features'],
            'training_samples': sample_model['metadata']['training_samples']
        }
        
        print(json.dumps(display_metadata, indent=2))

# 💾 Mount Google Drive

Mount Google Drive to save trained models for long-term storage.

In [ ]:
def mount_google_drive():
    """Mount Google Drive safely"""
    global DRIVE_MOUNTED
    
    try:
        from google.colab import drive
        print("🔄 Mounting Google Drive...")
        drive.mount('/content/drive')
        
        # Verify mount
        if os.path.exists('/content/drive/MyDrive'):
            print("✅ Google Drive mounted successfully")
            print(f"📁 Drive path: {MODEL_SAVE_DRIVE_PATH}")
            
            # Create models directory in Drive if it doesn't exist
            os.makedirs(MODEL_SAVE_DRIVE_PATH, exist_ok=True)
            DRIVE_MOUNTED = True
            return True
        else:
            print("❌ Drive mount verification failed")
            return False
            
    except ImportError:
        print("⚠️  Not running in Google Colab - Drive mount skipped")
        return False
    except Exception as e:
        print(f"⚠️  Drive mount failed: {e}")
        print("Continuing without Drive backup...")
        return False

# Mount Google Drive
mount_success = mount_google_drive()

if mount_success:
    print("💡 Models will be saved to both repo and Google Drive")
else:
    print("💡 Models will be saved to repo only")

# 📥 Import Modules

Import the trading bot modules and verify everything is working.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Core libraries
import pandas as pd
import numpy as np
import json
import joblib
from datetime import datetime, timedelta
from pathlib import Path
import hashlib
import subprocess

# ML libraries
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score

# Optional libraries (with fallbacks)
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
    print("✅ XGBoost available")
except ImportError:
    XGB_AVAILABLE = False
    print("⚠️  XGBoost not available - will skip XGBoost models")

try:
    import optuna
    OPTUNA_AVAILABLE = True
    print("✅ Optuna available")
except ImportError:
    OPTUNA_AVAILABLE = False
    print("⚠️  Optuna not available - will skip hyperparameter optimization")

# Set random seeds
np.random.seed(CFG['seed'])

print(f"\n🔧 Core libraries imported successfully")
print(f"🎯 Random seed: {CFG['seed']}")

In [ ]:
def import_trading_modules():
    """Import trading bot modules with fallbacks"""
    
    if not REPO_CLONED:
        print("❌ Repository not available for module imports")
        return False
    
    print("🔄 Importing trading bot modules...")
    
    # Try to import existing modules
    modules_imported = {}
    
    # Feature engineering
    try:
        from arbi.ai.feature_engineering_v2 import compute_features_deterministic, load_feature_schema
        modules_imported['feature_engineering'] = True
        print("✅ Feature engineering module")
    except ImportError as e:
        print(f"⚠️  Feature engineering module not found: {e}")
        modules_imported['feature_engineering'] = False
    
    # Training module
    try:
        from arbi.ai.training_v2 import generate_synthetic_ohlcv_data
        modules_imported['training'] = True
        print("✅ Training module")
    except ImportError:
        try:
            from arbi.ai.train_lgbm import train_and_validate_lgbm
            modules_imported['training'] = True
            print("✅ LightGBM training module")
        except ImportError as e:
            print(f"⚠️  Training module not found: {e}")
            modules_imported['training'] = False
    
    # Model registry
    try:
        from arbi.ai.registry import ModelRegistry
        modules_imported['registry'] = True
        print("✅ Model registry")
    except ImportError as e:
        print(f"⚠️  Model registry not found: {e}")
        modules_imported['registry'] = False
    
    # Inference module
    try:
        from arbi.ai.inference_v2 import ProductionInferenceEngine
        modules_imported['inference'] = True
        print("✅ Inference engine")
    except ImportError:
        try:
            from arbi.ai.inference import InferenceEngine
            modules_imported['inference'] = True
            print("✅ Inference engine (v1)")
        except ImportError as e:
            print(f"⚠️  Inference module not found: {e}")
            modules_imported['inference'] = False
    
    imported_count = sum(modules_imported.values())
    total_count = len(modules_imported)
    
    print(f"\n📊 Module Import Summary: {imported_count}/{total_count} modules imported")
    
    if imported_count == 0:
        print("⚠️  No trading bot modules found - will use fallback implementations")
        return False
    elif imported_count < total_count:
        print("⚠️  Some modules missing - will use fallbacks where needed")
        return True
    else:
        print("✅ All modules imported successfully")
        return True

# Import trading bot modules
modules_available = import_trading_modules()

# 🏋️ Model Training

Train LightGBM and XGBoost models using your existing modules or fallback implementations.

In [ ]:
def create_fallback_features(df):
    """Create basic technical indicators as fallback"""
    features = pd.DataFrame(index=df.index)
    
    # Price features
    features['returns'] = df['close'].pct_change()
    features['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    features['price_ma5'] = df['close'].rolling(5).mean()
    features['price_ma20'] = df['close'].rolling(20).mean()
    features['price_ratio_ma5'] = df['close'] / features['price_ma5']
    features['price_ratio_ma20'] = df['close'] / features['price_ma20']
    
    # Volume features
    features['volume_ma5'] = df['volume'].rolling(5).mean()
    features['volume_ratio'] = df['volume'] / features['volume_ma5']
    features['volume_price_trend'] = features['volume_ratio'] * features['returns']
    
    # Volatility
    features['volatility'] = features['returns'].rolling(20).std()
    features['volatility_ratio'] = features['returns'].abs() / features['volatility']
    
    # RSI
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    features['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD
    exp1 = df['close'].ewm(span=12).mean()
    exp2 = df['close'].ewm(span=26).mean()
    features['macd'] = exp1 - exp2
    features['macd_signal'] = features['macd'].ewm(span=9).mean()
    features['macd_hist'] = features['macd'] - features['macd_signal']
    
    return features.dropna()

def generate_fallback_ohlcv_data(n_periods=1000, symbol="BTC-USD"):
    """Generate synthetic OHLCV data"""
    dates = pd.date_range(start='2023-01-01', periods=n_periods, freq='1H')
    
    # Random walk with drift and regime changes
    np.random.seed(CFG['seed'])
    
    # Create regime changes
    regime_changes = np.random.choice(n_periods, size=5, replace=False)
    regime_changes.sort()
    
    returns = []
    current_vol = 0.01
    
    for i in range(n_periods):
        # Change volatility at regime boundaries
        if i in regime_changes:
            current_vol = np.random.uniform(0.005, 0.02)
        
        # Generate return with current volatility
        ret = np.random.normal(0.00005, current_vol)
        returns.append(ret)
    
    returns = np.array(returns)
    prices = 50000 * np.exp(np.cumsum(returns))
    
    data = []
    for i, (date, price) in enumerate(zip(dates, prices)):
        high = price * (1 + abs(np.random.normal(0, 0.005)))
        low = price * (1 - abs(np.random.normal(0, 0.005)))
        open_price = prices[i-1] if i > 0 else price
        volume = np.random.uniform(100, 1000) * (1 + abs(returns[i]) * 10)
        
        data.append({
            'timestamp': date,
            'open': open_price,
            'high': high,
            'low': low,
            'close': price,
            'volume': volume
        })
    
    return pd.DataFrame(data)

def create_training_dataset(n_periods, symbol):
    """Create training dataset with features and labels"""
    
    print(f"🔄 Creating training dataset ({n_periods} periods)...")
    
    # Generate or load OHLCV data
    try:
        if modules_available:
            from arbi.ai.training_v2 import generate_synthetic_ohlcv_data
            df = generate_synthetic_ohlcv_data(n_periods, symbol)
            print("✅ Using repository OHLCV generation")
        else:
            raise ImportError("Using fallback")
    except:
        df = generate_fallback_ohlcv_data(n_periods, symbol)
        print("✅ Using fallback OHLCV generation")
    
    # Compute features
    try:
        if modules_available:
            from arbi.ai.feature_engineering_v2 import compute_features_deterministic
            feature_result = compute_features_deterministic(df, symbol)
            feature_df = feature_result.features
            print("✅ Using repository feature engineering")
        else:
            raise ImportError("Using fallback")
    except:
        feature_df = create_fallback_features(df)
        print("✅ Using fallback feature engineering")
    
    # Create labels
    future_periods = CFG['horizon']
    threshold = CFG['pos_thresh']
    
    # Calculate future returns
    future_returns = df['close'].shift(-future_periods) / df['close'] - 1
    
    # Binary classification: 1 if return > threshold, 0 otherwise
    labels_binary = (future_returns > threshold).astype(int)
    
    # Regression target: actual future return
    labels_regression = future_returns
    
    # Remove rows where we can't calculate future returns
    valid_mask = ~future_returns.isna()
    
    feature_df = feature_df[valid_mask].reset_index(drop=True)
    labels_binary = labels_binary[valid_mask].reset_index(drop=True)
    labels_regression = labels_regression[valid_mask].reset_index(drop=True)
    timestamps = df['timestamp'][valid_mask].reset_index(drop=True)
    
    print(f"✅ Dataset created:")
    print(f"  Samples: {len(feature_df)}")
    print(f"  Features: {feature_df.shape[1]}")
    print(f"  Positive class: {labels_binary.sum()}/{len(labels_binary)} ({100*labels_binary.mean():.1f}%)")
    print(f"  Regression target range: {labels_regression.min():.4f} to {labels_regression.max():.4f}")
    
    return feature_df, labels_binary, labels_regression, timestamps

# Create training dataset
X, y_binary, y_regression, timestamps = create_training_dataset(CFG['n_periods'], SYMBOL)

# 🎉 Notebook Complete!

This is the complete Google Colab training notebook. To continue with training and saving models, use the additional chunks or run the CLI script.

**Next steps:**
1. Run the remaining training cells
2. Save model artifacts
3. Create training manifest
4. Download results

**Or use the CLI script:** `python tools/colab_train.py`